In [30]:
!pip install -U albumentations
!pip install opencv-python
!pip install torchvision

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
from torchvision import datasets, transforms

In [25]:
# Define the path to your dataset
dataset_path = "./sea-turtles-1-test/all/images/"

In [26]:
# Define a set of augmentations
transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
    A.Rotate(limit=30),
    A.RandomBrightnessContrast(),
    A.Normalize(),
    ToTensorV2(),
])

In [29]:
# Use torchvision's ImageFolder to load the dataset
# ImageFolder assumes the dataset is organized with subfolders for each class
# and each image in its respective class folder.
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

FileNotFoundError: Couldn't find any class folder in ./sea-turtles-1-test/all/images/.

In [28]:
# Create a DataLoader to efficiently load and batch the data during training
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Accessing dataset labels
class_labels = dataset.classes

NameError: name 'dataset' is not defined

In [33]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define paths to the image and annotation directories
image_dir = "./sea-turtles-1-test/all/images/"
annotation_dir = "./sea-turtles-1-test/all/labels/"

# Define the transformation pipeline including image and bounding box augmentations
transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
    A.Rotate(limit=30),
    A.RandomBrightnessContrast(),
    A.Normalize(),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Get the list of image files
image_files = os.listdir(image_dir)

# Iterate through each image
for image_file in image_files:
    # Load the image
    image_path = os.path.join(image_dir, image_file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load the corresponding annotation file
    annotation_file = os.path.join(annotation_dir, image_file.replace('.jpg', '.txt'))
    with open(annotation_file, 'r') as f:
        # Assuming each line of the annotation file contains [class, xmin, ymin, xmax, ymax]
        annotation = [float(coord) for coord in f.readline().strip().split()]

    # Extract the bounding box coordinates
    class_label, xmin, ymin, xmax, ymax = annotation

    # Apply the augmentation to the image and bounding box
    augmented = transform(image=image, bboxes=[[xmin, ymin, xmax, ymax]], class_labels=[class_label])

    # Extract the augmented image and bounding box
    augmented_image = augmented['image']
    augmented_bbox = augmented['bboxes'][0]

    # Update the annotation file with the new bounding box coordinates
    with open(annotation_file, 'w') as f:
        f.write(f"{class_label} {augmented_bbox[0]} {augmented_bbox[1]} {augmented_bbox[2]} {augmented_bbox[3]}")

    # Save the augmented image
    augmented_image_path = os.path.join("./sea-turtles-1-test/all/augmented_images/", image_file)
    cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))


ValueError: not enough values to unpack (expected 5, got 0)